# Taller de SQLite

## 1. Introducción

Este taller práctico te guiará a través de la creación de una base de datos SQLite para gestionar una empresa, cubriendo:

- Creación de tablas y relaciones (DDL)

- Inserción y manipulación de datos (DML)

- Consultas avanzadas (JOINS, GROUP BY, HAVING, subconsultas)

- Control de acceso y permisos (DCL)

- Visualización del esquema de la base de datos
---

## 2. Creación de la Base de Datos y Tablas (DDL)

In [1]:
import sqlite3

#conectar a la base de datos
conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

In [2]:
# crear tablas
cursor.execute("""
CREATE TABLE IF NOT EXISTS departamentos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS empleados (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    edad INTEGER,
    departamento_id INTEGER,
    FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS proyectos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    cliente_id INTEGER,
    FOREIGN KEY (cliente_id) REFERENCES clientes(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS facturas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    proyecto_id INTEGER,
    monto REAL NOT NULL,
    fecha TEXT NOT NULL,
    FOREIGN KEY (proyecto_id) REFERENCES proyectos(id)
)
""")

conn.commit()
conn.close()

## 3. Inserción de Datos (DML)

In [3]:
# Insercion de datos en lotes

conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

# datos en departamentos
cursor.executemany("INSERT INTO departamentos (nombre) VALUES (?)", [
    ('Ventas',), ('IT',), ('Recursos Humanos',)
])

# empleados
cursor.executemany("INSERT INTO empleados (nombre, edad, departamento_id) VALUES (?, ?, ?)", [
    ('Ana Pérez', 30, 1), ('Carlos López', 28, 2), ('María García', 35, 3)
])

# clientes
cursor.executemany("INSERT INTO clientes (nombre, email) VALUES (?, ?)", [
    ('Empresa XYZ', 'contacto@xyz.com'),
    ('Tech Solutions', 'info@techsolutions.com')
])

# proyectos
cursor.executemany("INSERT INTO proyectos (nombre, cliente_id) VALUES (?, ?)", [
    ('App Movil', 1), ('Plataforma Web', 2)
])

# facturas
cursor.executemany("INSERT INTO facturas (proyecto_id, monto, fecha) VALUES (?, ?, ?)", [
    (1, 1500.00, '2025-04-01'),
    (2, 3000.00, '2025-04-02')
])

conn.commit()
conn.close()

In [4]:
# Inserción de datos individuales

conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

cursor.execute("INSERT INTO empleados (nombre, edad, departamento_id) VALUES ('Juan Torres', 40, 1)")
cursor.execute("INSERT INTO clientes (nombre, email) VALUES ('Innovatech', 'contacto@innovatech.com')")
cursor.execute("INSERT INTO proyectos (nombre, cliente_id) VALUES ('Sistema de Facturación', 3)")
cursor.execute("INSERT INTO facturas (proyecto_id, monto, fecha) VALUES (3, 5000.00, '2025-04-03')")

conn.commit()
conn.close()

## 4. Consultas Avanzadas

In [5]:

#4.1 Listar empleados con sus departamentos
conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()
cursor.execute("""
SELECT empleados.nombre, empleados.edad, departamentos.nombre AS departamento
FROM empleados
JOIN departamentos ON empleados.departamento_id = departamentos.id
""")
print(cursor.fetchall())

#4.2 Obtener total facturado por cliente

cursor.execute("""
SELECT clientes.nombre, SUM(facturas.monto) AS total_facturado
FROM clientes
JOIN proyectos ON clientes.id = proyectos.cliente_id
JOIN facturas ON proyectos.id = facturas.proyecto_id
GROUP BY clientes.id
""")
print(cursor.fetchall())

#4.3 Obtener proyectos con facturas mayores a $2000

cursor.execute("""
SELECT proyectos.nombre, facturas.monto
FROM proyectos
JOIN facturas ON proyectos.id = facturas.proyecto_id
WHERE facturas.monto > 2000
""")
print(cursor.fetchall())

#4.4 Contar la cantidad de empleados por departamento

cursor.execute("""
SELECT departamentos.nombre, COUNT(empleados.id) AS cantidad_empleados
FROM departamentos
LEFT JOIN empleados ON departamentos.id = empleados.departamento_id
GROUP BY departamentos.id
""")
print(cursor.fetchall())

#4.5 Obtener los proyectos con sus respectivos clientes

cursor.execute("""
SELECT proyectos.nombre AS proyecto, clientes.nombre AS cliente
FROM proyectos
JOIN clientes ON proyectos.cliente_id = clientes.id
""")
print(cursor.fetchall())

#4.6 Listar facturas ordenadas por monto de mayor a menor

cursor.execute("""
SELECT * FROM facturas
ORDER BY monto DESC
""")

print(cursor.fetchall())
conn.close()

[('Ana Pérez', 30, 'Ventas'), ('Carlos López', 28, 'IT'), ('María García', 35, 'Recursos Humanos'), ('Juan Torres', 40, 'Ventas')]
[('Empresa XYZ', 1500.0), ('Tech Solutions', 3000.0), ('Innovatech', 5000.0)]
[('Plataforma Web', 3000.0), ('Sistema de Facturación', 5000.0)]
[('Ventas', 2), ('IT', 1), ('Recursos Humanos', 1)]
[('App Movil', 'Empresa XYZ'), ('Plataforma Web', 'Tech Solutions'), ('Sistema de Facturación', 'Innovatech')]
[(3, 3, 5000.0, '2025-04-03'), (2, 2, 3000.0, '2025-04-02'), (1, 1, 1500.0, '2025-04-01')]


In [ ]:
import pandas as pd
conn = sqlite3.connect("empresa.db")
df = pd.read_sql_query("SELECT * FROM empleados", conn)
conn.close()
df.head()

,id,nombre,edad,departamento_id
0,1,Ana Pérez,30,1
1,2,Carlos López,28,2
2,3,María García,35,3
3,4,Juan Torres,40,1


## 5. Visualización del Esquema

---
## 6. Actividad

- Agregar una tabla "Tareas" asociada a empleados y proyectos.

- Consultar la cantidad de empleados por departamento.

- Actualizar la edad de un empleado.

- Eliminar un cliente y analizar el efecto en los proyectos asociados.